<a href="https://colab.research.google.com/github/GabrielLoiseau/tau-eval/blob/main/examples/Tau_Eval_Authorship_Use_Case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U tau-eval datasets huggingface_hub fsspec

# Use Case: Authorship Obfuscation with Tau-Eval

**Introduction** Authorship obfuscation aims to modify a piece of text to conceal the original author's writing style, thereby protecting their privacy or anonymizing the text for other purposes. This can involve techniques like paraphrasing, style transfer, or simplification.

This notebook demonstrates how to use Tau-Eval's built-in models designed for authorship modification/obfuscation. We will also explore how to use Tau-Eval's utility metrics and visualization tools to assess the impact of these transformations on the text's content and quality.

## Section 1: Setting up Environment and Models

In [2]:
import os
# Only used if you evaluate api-based models
os.environ["OPENROUTER_API_KEY"] = "your-api-key"

In [ ]:
from tau_eval.models.authorship import KeepItSimple, Paraphraser, M2M100MT
#from tau_eval.models.llm_api import LLMAuthorship

p1 = KeepItSimple()
p2 = Paraphraser()
p3 = M2M100MT()
# Uncomment if you are using an Openrouter API key
#p4 = LLMAuthorship("google/gemini-flash-1.5-8b")
#p5 = LLMAuthorship("google/gemini-flash-1.5")

MODEL_LIST = [p1,p2,p3]

## Section 2: Preparing Tasks and Metrics

In [ ]:
from tau_eval.tasks import IMDBAuthorshipClassification
import tasknet as tn

imdb = IMDBAuthorshipClassification(n_authors=10, max_rows=1000, max_rows_eval=1000)
dynahate = tn.AutoTask('dynahate', max_rows=10000, max_rows_eval=1000)
anli = tn.AutoTask('anli/a1')

METRICS = ["rouge","meteor","bertscore"]
TASKS = [imdb,dynahate,anli]

## Section 3: Running the Authorship Obfuscation Experiment

In [ ]:
from tau_eval import Experiment, ExperimentConfig

config = ExperimentConfig("authorship-obfuscation","answerdotai/ModernBERT-base",True, False)

e = Experiment(MODEL_LIST,METRICS,TASKS,config)

e.run(output_dir="results-authorship.json")

## Section 4: Visualize results

In [ ]:
from tau_eval import Experiment

e = Experiment.from_json("results-authorship.json")

In [ ]:
e.summary().keys()

In [ ]:
from tau_eval.visualization import plot_all_metrics_for_model_on_dataset

plot_all_metrics_for_model_on_dataset(e.results, 'anli/plain_text_2', p1.name)

In [ ]:
from tau_eval.visualization import plot_metric_distribution

plot_metric_distribution(e.results, 'rouge1')

In [ ]:
from tau_eval.visualization import plot_trade_off_metric

plot_trade_off_metric(e.results, "test_accuracy", "rouge1", ['anli/plain_text_2'], [p.name for p in MODEL_LIST])